In [2]:
import google.colab as colab
from google.colab import drive 
colab.drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [48]:
with open("/content/drive/MyDrive/NeuralNetwork/faust.txt", "r", encoding="utf-8") as file:
  contents = file.read()

contents = contents.split("\n")[52:7052]
contents = [line.strip() for line in contents]

contents = "\n".join(contents)

contents = contents.replace("\n", " \\n ")

In [ ]:
!pip install nltk sklearn

In [49]:
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')

tokens = word_tokenize(contents)

# Can be used to use the words which are used most often.
# For example: if max_features is set to 1000 it will build an array from the first 1000 words which have the highest amount of occurrences.
cv = CountVectorizer(max_features=len(set(tokens)), lowercase=False, token_pattern="(.*)")
cv.fit(tokens)

features = cv.get_feature_names_out()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
word_to_int = {}
int_to_word = {}

for i in range(0, len(features)):
    word = features[i]
    
    word_to_int[word] = i
    int_to_word[i] = word

print(word_to_int['Herz'])
print(int_to_word[1284])
print(int_to_word)

1284
Herz
{0: '', 1: '!', 2: "'", 3: "''", 4: "'raus", 5: "'s", 6: '(', 7: ')', 8: ',', 9: '-', 10: '--', 11: '.', 12: '..', 13: ':', 14: ';', 15: '?', 16: 'A', 17: 'ALLE', 18: 'ALTE', 19: 'ALTER', 20: 'ALTMAYER', 21: 'ANDERE', 22: 'ANDRE', 23: 'ANDRER', 24: 'ARIEL', 25: 'AUTOR', 26: 'Aas', 27: 'Ab', 28: 'Abend', 29: 'Abendrot', 30: 'Abendsonne-Glut', 31: 'Abendstrahl', 32: 'Aber', 33: 'Abgeschmackters', 34: 'Abgrund', 35: 'Abgrunds', 36: 'Abscheu', 37: 'Ach', 38: 'Acht', 39: 'Acker', 40: 'Adams', 41: 'Ade', 42: 'Adepten', 43: 'Adern', 44: 'Adler', 45: 'Affe', 46: 'Affen', 47: 'Agathe', 48: 'Ah', 49: 'Aha', 50: 'Ahnen', 51: 'Ahnherrn', 52: 'Ahnung', 53: 'Ahnungsdrang', 54: 'Akkorden', 55: 'All', 56: 'Alle', 57: 'Allein', 58: 'Allem', 59: 'Allerhalter', 60: 'Allerschlimmste', 61: 'Alles', 62: 'Allmächtiger', 63: 'Allumfasser', 64: 'Allwissend', 65: 'Alpenfeld', 66: 'Als', 67: 'Altar', 68: 'Alte', 69: 'Alten', 70: 'Alter', 71: 'Altklug', 72: 'Am', 73: 'Amt', 74: 'An', 75: 'Anblick', 76: 

In [51]:
int_tokens = []

for i in range(0, len(tokens)):
  if (tokens[i] in features):
    int_tokens.append(word_to_int[tokens[i]])

In [52]:
import numpy as np

x = []
y = []

seq_length = 40

for i in range(0, len(int_tokens) - seq_length):
    x.append(int_tokens[i:i+seq_length])
    y.append(int_tokens[i+seq_length])
    
x = np.array(x)
y = np.array(y)

In [52]:
x[0]

array([  11, 3419, 3419, 1416, 5479, 4257, 6728,    8, 5884, 1057,    8,
       3419,  460, 4407, 5975, 4071, 3898, 6254,  286, 4670,   11, 3419,
       3057, 5014, 6771,    8, 4257, 3938, 4314,   15, 3419,  884, 5014,
       5391, 1284, 5535, 5078, 3126, 4565,   15])

In [53]:
# Validation of x and y

txt = []
for i in range(0, len(x[0])):
  txt.append(int_to_word[x[0][i]])

print(txt)

print(y[0])
print(int_to_word[y[0]])

['Zueignung', '.', '\\n', '\\n', 'Ihr', 'naht', 'euch', 'wieder', ',', 'schwankende', 'Gestalten', ',', '\\n', 'Die', 'früh', 'sich', 'einst', 'dem', 'trüben', 'Blick', 'gezeigt', '.', '\\n', 'Versuch', 'ich', 'wohl', ',', 'euch', 'diesmal', 'festzuhalten', '?', '\\n', 'Fühl', 'ich', 'mein', 'Herz', 'noch', 'jenem', 'Wahn', 'geneigt']
15
?


In [54]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, Flatten

model = Sequential()
# Embedding layer to find similarity of words
model.add(Embedding(cv.max_features, 150, input_shape=(seq_length,)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256, return_sequences=False))

model.add(Dense(1000, activation="sigmoid"))

model.add(Dense(cv.max_features, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [56]:
from keras.utils import to_categorical

# Required because output layer gives % of probability of word
y_categorical = to_categorical(y, num_classes=cv.max_features)

model.fit(x, y_categorical, epochs=10, batch_size=32)

Epoch 1/10
1418/1418 [==============================] - 36s 23ms/step - loss: 6.0543 - accuracy: 0.1820
Epoch 2/10
1418/1418 [==============================] - 19s 13ms/step - loss: 5.2162 - accuracy: 0.2265
Epoch 3/10
1418/1418 [==============================] - 17s 12ms/step - loss: 4.8910 - accuracy: 0.2354
Epoch 4/10
1418/1418 [==============================] - 16s 11ms/step - loss: 4.6897 - accuracy: 0.2445
Epoch 5/10
1418/1418 [==============================] - 15s 11ms/step - loss: 4.5364 - accuracy: 0.2553
Epoch 6/10
1418/1418 [==============================] - 16s 11ms/step - loss: 4.4002 - accuracy: 0.2666
Epoch 7/10
1418/1418 [==============================] - 15s 11ms/step - loss: 4.2792 - accuracy: 0.2783
Epoch 8/10
1418/1418 [==============================] - 16s 11ms/step - loss: 4.1594 - accuracy: 0.2887
Epoch 9/10
1418/1418 [==============================] - 16s 11ms/step - loss: 4.0467 - accuracy: 0.2988
Epoch 10/10
1418/1418 [==============================] - 15s 11m

In [85]:
model.save("/content/drive/MyDrive/NeuralNetwork/faust.model")

In [86]:
import pickle

with open("/content/drive/MyDrive/NeuralNetwork/word_to_int.faust.pickle", "wb") as file:
    pickle.dump(word_to_int, file)
    
    
with open("/content/drive/MyDrive/NeuralNetwork/int_to_word.faust.pickle", "wb") as file:
    pickle.dump(int_to_word, file)

In [87]:
# Load trained model and dictionary

from keras.models import load_model
import pickle

model = load_model("/content/drive/MyDrive/NeuralNetwork/faust.model")

with open("/content/drive/MyDrive/NeuralNetwork/word_to_int.faust.pickle", "rb") as file:
    word_to_int = pickle.load(file)
    
with open("/content/drive/MyDrive/NeuralNetwork/int_to_word.faust.pickle", "rb") as file:
    int_to_word = pickle.load(file)

In [89]:
import numpy as np

sentence = np.array(int_tokens[100:140])
txt = ''

for i in range(0, 100):
    prediction = model.predict(sentence.reshape(1, 40))
    
    # word = np.argmax(prediction[0])
    word = np.random.choice(len(int_to_word), p=prediction[0])
    txt = txt + int_to_word[word].replace("\\n", "\n")
    if (int_to_word[word] != "\\n"):
      txt = txt + " "
    sentence = np.append(sentence[1:], [word])

print(txt)

1/1 [==============================] - 0s 19ms/step
die bist und Väterthron am Busen wär , 
Daß er ins Raum tot mir durch immer gewesen , 
Nicht ein nur spielen es nicht wart ! 

Mein wußte bald sie sein exerzieren . 
Rette von nur auch nach Aug auf mit die Mann , 
Und Er die Jungens Für der Die Baum ? 

MARTHE : 
Dahinaus : 
Was PERSON : Juchhe ! Die bißchen hast mit wider herum- . 
So war hohl zum war sonst geschenkt . 
Muß ihr schon Margareten ? 
O irr und gut , 
